In [1]:
from pyspark.sql import SparkSession

In [2]:
!pwd
!ls

/home/jovyan/work
postgresql-42.5.1.jar  sales_etl.ipynb


In [3]:
spark = SparkSession.builder\
                    .appName("salesETL")\
                    .config("spark.driver.extraClassPath", "./postgresql-42.5.1.jar")\
                    .getOrCreate()

In [4]:
df = spark.read.format("jdbc").option("url", "jdbc:postgresql://postgres_container:5432/postgres")\
                              .option("dbtable", "sales")\
                              .option("driver", "org.postgresql.Driver")\
                              .option("user", "admin")\
                              .option("password", "admin").load()

In [5]:
df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_lastname: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- order_value: double (nullable = true)



In [7]:
df.show(10).toPandas()

+--------+----------+-----------+-------------+-----------------+-------------+--------------+----------+--------+-----------+
|order_id|order_date|customer_id|customer_name|customer_lastname|customer_city|customer_state|product_id|quantity|order_value|
+--------+----------+-----------+-------------+-----------------+-------------+--------------+----------+--------+-----------+
+--------+----------+-----------+-------------+-----------------+-------------+--------------+----------+--------+-----------+



AttributeError: 'NoneType' object has no attribute 'toPandas'